In [1]:
import pandas as pd
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 50)

In [2]:
# Data for dual language programs
df = pd.read_csv('data/dual_language.csv')

# School identification codes
dfs = pd.read_excel('data/schools.xls')
dfs.columns = map(str.lower, dfs.columns)
dfs = dfs[dfs['location type description'] == 'General Academic']
dfs = dfs.iloc[:, :23]
dfs = dfs.drop(dfs.iloc[:, [8,10,12,16,17,18]].columns, axis=1)

# Test scores
dfe = pd.read_csv('data/Annual EM ELA.csv')
dfe.columns = map(str.lower, dfe.columns)
dfe = dfe[dfe['year'] == 2022]
dfe = dfe[dfe['assessment_name'] == 'ELA5']
dfe = dfe.drop(dfe[dfe['mean_score'].isna()].index)
dfe = dfe.drop(dfe[dfe['mean_score'] == 's'].index)
dfe = dfe.drop(dfe[dfe['institution_id'].isna()].index)
dfe['institution_id'] = dfe['institution_id'].astype(int)
dfe['entity_name'] = dfe['entity_name'].str.lower()

# Test scores
dfm = pd.read_csv('data/Annual EM MATH.csv')
dfm.columns = map(str.lower, dfm.columns)
dfm = dfm[dfm['year'] == 2022]
dfm = dfm[dfm['assessment_name'] == 'MATH5']
dfm = dfm.drop(dfm[dfm['mean_score'].isna()].index)
dfm = dfm.drop(dfm[dfm['mean_score'] == 's'].index)
dfm = dfm.drop(dfm[dfm['institution_id'].isna()].index)
dfm['institution_id'] = dfm['institution_id'].astype(int)
dfm['entity_name'] = dfm['entity_name'].str.lower()
dfm = dfm.loc[:, ['entity_cd', 'per_prof', 'mean_score']]

In [3]:
# Restricting to schools that include kindergarten
dfs = dfs[dfs['grades'].str.contains('0K').fillna(False)]

In [4]:
# Restricting data to Brooklyn and Manhattan
target_districts = [1,2,3,13,14,15,16,17,18,19,20,21,22,23]
dfs = dfs[dfs['geographical district code'].isin(target_districts)]

In [5]:
# Selecting Spanish programs
df = df[df['Language'] == 'Spanish']

# Removing transitional language programs
df = df[df['Program'] == 'Dual Language']

# A few dual language Spanish schools do not appear in the test score data
dfs.loc[dfs[dfs['ats system code'].isin(df['School Code'].values)].index, 'spanish'] = True
spanish_codes = set(df['School Code'])
merged_spanish_codes = set(dfs[dfs['spanish'].fillna(False)]['ats system code'])
len(spanish_codes - merged_spanish_codes)

160

In [6]:
# Dual languaged Spanish schools that do not have test score
# Manually reviewed on insideschools.org and all are at or below median test scores
local_districts = [13,15,17,20]
dfna = df[df['School Code'].isin(spanish_codes - merged_spanish_codes)]
dfna = dfna[dfna['District'].isin(local_districts)]
dfna

,School Code,District,School Name,School Category,Program,Language,General/Special Education,Special Education Model,Principal Name,Principal Email
239,13K113,13,M.S. 113 Ronald Edmonds Learning Center,Junior High-Intermediate-Middle,Dual Language,Spanish,General Education,Not Applicable,Dawnique Daughtry,DDaught@schools.nyc.gov
244,13K691,13,Fort Greene Preparatory Academy,Junior High-Intermediate-Middle,Dual Language,Spanish,General Education,Not Applicable,Paula Lettiere,PLettie2@schools.nyc.gov
275,15K088,15,J.H.S. 088 Peter Rouget,Junior High-Intermediate-Middle,Dual Language,Spanish,General Education,Not Applicable,AILENE MITCHELL,AMitche2@schools.nyc.gov
282,15K136,15,I.S. 136 Charles O. Dewey,Junior High-Intermediate-Middle,Dual Language,Spanish,General Education,Not Applicable,Amanda Bueno,ABueno3@schools.nyc.gov
283,15K136,15,I.S. 136 Charles O. Dewey,Junior High-Intermediate-Middle,Dual Language,Spanish,Special Education,ICT,Amanda Bueno,ABueno3@schools.nyc.gov
290,15K516,15,Sunset Park Avenues Elementary School,Elementary,Dual Language,Spanish,General Education,Not Applicable,JESSICA KNUDSON,JKnudson2@schools.nyc.gov
293,15K667,15,Sunset Park High School,High school,Dual Language,Spanish,General Education,Not Applicable,Victoria Antonini,VAntoni2@schools.nyc.gov
294,15K667,15,Sunset Park High School,High school,Dual Language,Spanish,Special Education,ICT,Victoria Antonini,VAntoni2@schools.nyc.gov
297,15K896,15,The School of Creativity and Innovation,Elementary,Dual Language,Spanish,General Education,Not Applicable,RUTH RODRIGUEZ,RRodriguez@schools.nyc.gov
309,17K352,17,Ebbets Field Middle School,Junior High-Intermediate-Middle,Dual Language,Spanish,Special Education,ICT,SHERY ANN ATKINSON,SAtkinson@schools.nyc.gov


In [7]:
# Merging test scores
dfs = dfs.merge(dfe, how='left', left_on='beds number', right_on='entity_cd')
dfs = dfs.iloc[:, [2,3,7,10,13,17,25,29,39,41]]
dfs = dfs.merge(dfm, how='left', left_on='beds number', right_on='entity_cd',
         suffixes=('_e', '_m'))
dfs = dfs.drop(['beds number', 'entity_cd', 'city'], axis=1)
dfs = dfs.fillna(0)
dfs['mean_score_m'] = dfs['mean_score_m'].astype(int)
dfs = dfs.sort_values('mean_score_m', ascending=False)

In [8]:
# Dual language Spanish programs sorted by language arts test score
dfs[dfs['spanish'] == True].head(5)

,location name,grades,geographical district code,spanish,total_count,pct_tested,per_prof_e,mean_score_e,per_prof_m,mean_score_m
53,P.S. 087 William Sherman,"0K,01,02,03,04,05,SE",3,True,91.0,97.0,68,616,71,610
115,P.S. 094 The Henry Longfellow,"0K,01,02,03,04,05,SE",15,True,163.0,92.0,42,605,61,609
50,P.S. 075 Emily Dickinson,"0K,01,02,03,04,05,SE",3,True,70.0,84.0,66,612,58,608
183,P.S. 108 Sal Abbracciamento,"0K,01,02,03,04,05,SE",19,True,120.0,96.0,45,605,65,608
52,P.S. 084 Lillian Weber,"0K,01,02,03,04,05,SE",3,True,51.0,96.0,80,619,60,608


In [9]:
# All schools in Brooklyn and Manhattan sorted by math test score
dfs.head(20)

,location name,grades,geographical district code,spanish,total_count,pct_tested,per_prof_e,mean_score_e,per_prof_m,mean_score_m
29,P.S. 77 Lower Lab School,"0K,01,02,03,04,05,SE",2,0,50.0,98.0,94,632,100,638
17,"New Explorations into Science, Technology and ...","0K,01,02,03,04,05,06,07,08,09,10,11,12",1,0,104.0,100.0,94,631,100,637
66,The Anderson School,"0K,01,02,03,04,05,06,07,08",3,0,53.0,100.0,100,634,98,634
36,P.S. 150,"0K,01,02,03,04,05",2,0,19.0,95.0,89,624,94,628
62,P.S. 199 Jessie Isador Straus,"0K,01,02,03,04,05,SE",3,0,88.0,98.0,84,621,87,621
123,P.S. 172 Beacon School of Excellence,"0K,01,02,03,04,05,SE",15,0,81.0,51.0,95,624,93,621
33,P.S. 124 Yung Wing,"0K,01,02,03,04,05,SE",2,0,95.0,99.0,82,620,88,620
91,P.S. 031 Samuel F. Dupont,"0K,01,02,03,04,05,SE",14,0,84.0,71.0,92,626,84,620
67,Special Music School,"0K,01,02,03,04,05,06,07,08",3,0,14.0,100.0,86,628,93,619
226,BROOKLYN SCHOOL OF INQUIRY,"0K,01",20,0,54.0,100.0,81,618,79,619


In [10]:
# Schools in districts 13,15,17 and 20 sorted by math test score
dfs = dfs[dfs['geographical district code'].isin(local_districts)]
dfs.sort_values('mean_score_m', ascending=False).head(40)

,location name,grades,geographical district code,spanish,total_count,pct_tested,per_prof_e,mean_score_e,per_prof_m,mean_score_m
123,P.S. 172 Beacon School of Excellence,"0K,01,02,03,04,05,SE",15,0,81.0,51.0,95,624,93,621
226,BROOKLYN SCHOOL OF INQUIRY,"0K,01",20,0,54.0,100.0,81,618,79,619
205,P.S. 105 The Blythebourne,"PK,0K,01,02,03,04,05,SE",20,0,202.0,96.0,58,610,76,616
113,P.S. 039 Henry Bristow,"0K,01,02,03,04,05",15,0,55.0,84.0,83,620,84,616
116,P.S. 107 John W. Kimball,"PK,0K,01,02,03,04,05,SE",15,0,74.0,68.0,74,612,80,614
127,P.S. 321 William Penn,"PK,0K,01,02,03,04,05,SE",15,0,184.0,81.0,73,617,76,614
211,Ralph A. Fabrizio School,"0K,01,02,03,04,05,SE",20,0,154.0,88.0,59,610,72,612
219,P.S. 204 Vince Lombardi,"0K,01,02,03,04,05,SE",20,0,198.0,96.0,66,613,68,611
202,P.S. 69 Vincent D. Grippo School,"0K,01,02,03,04,05,SE",20,0,116.0,97.0,68,614,66,611
71,P.S. 011 Purvis J. Behan,"PK,0K,01,02,03,04,05,06,SE",13,0,125.0,85.0,71,615,65,610


In [ ]:
# P.S. 039 Henry Bristow